In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the pre-trained layers
for layer in vgg16.layers:
    layer.trainable = False

# Add new classification layers on top of the pre-trained model
model = tf.keras.Sequential([
    vgg16,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 64)                524352    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 15,239,300
Trainable params: 524,612
Non-trainable params: 14,714,688
_________________________________________________________________


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
# Load the data

train_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'DATASETS/Alzheimer_s Dataset/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        'DATASETS/Alzheimer_s Dataset/test',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [ ]:

from PIL import Image

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

history = model.fit_generator(train_generator,
      steps_per_epoch=100,
      epochs=1,
      #callbacks=[callback],
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
val_acc = history.history
print(val_acc)

In [11]:
model.save("vgg16.h5")

In [ ]:
from numpy import argmax
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import load_model
from PIL import Image
import numpy as np

filename = 'DATASETS/Alzheimer_s Dataset/test/VeryMildDemented/26 (44).jpg'   
model = load_model('vgg16.h5')
img = load_img(filename, target_size=(150, 150))
img = img_to_array(img)
img = img.astype('float32')
img = img / 255.0
print(img.shape)
img = np.expand_dims(img,axis=0)
Image.open(filename).show()
predict_value = np.squeeze(model.predict(img))
print(predict_value)
